In [12]:
!pip install -q langchain langchain_community crewai langchain-openai tavily-python

In [13]:
!pip install crewai[tools]

In [14]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [15]:
from pydantic import BaseModel, Field
from typing import List
from datetime import date

class SupplyChainStepInput(BaseModel):
    step_name: str = Field(..., description="Name of the supply chain step")
    step_type: str = Field(..., description="Type of the supply chain step (e.g., 'Procurement', 'Manufacturing', 'Distribution', 'Retail')")
    processing_time: float = Field(..., description="Time taken to process at this step (in hours)")
    wait_time: float = Field(..., description="Wait time before this step begins (in hours)")
    transportation_time: float = Field(..., description="Transportation time associated with this step (in hours)")
    inventory_level: int = Field(..., description="Inventory level at this step")
    demand_rate: int = Field(..., description="Demand rate at this step (units per day)")
    supply_rate: int = Field(..., description="Supply rate at this step (units per day)")
    holding_cost: float = Field(..., description="Inventory holding cost at this step (per unit per day)")
    stockout_cost: float = Field(..., description="Stockout cost at this step (per unit)")
    resources: List[str] = Field(..., description="Resources used in this step (e.g., 'Warehouse A', 'Fleet 1')")

class SupplyChainInputData(BaseModel):
    chain_name: str = Field(..., description="Name of the supply chain")
    date_created: date = Field(..., description="Date when the analysis is created")
    steps: List[SupplyChainStepInput] = Field(..., description="List of supply chain steps with initial data")

class SupplyChainStep(SupplyChainStepInput):
    cycle_time: float = Field(..., description="Total cycle time at this step (processing + wait + transportation time) in hours")
    throughput_rate: float = Field(..., description="Throughput rate at this step (units per day)")
    work_in_progress: int = Field(..., description="Work-in-progress inventory at this step")
    total_inventory_cost: float = Field(..., description="Total inventory holding cost at this step")
    total_stockout_cost: float = Field(..., description="Total stockout cost at this step")
    total_step_cost: float = Field(..., description="Total cost at this step (holding cost + stockout cost)")
    utilization: float = Field(..., description="Utilization rate of resources at this step (percentage)")

class SupplyChainAnalysis(BaseModel):
    chain_name: str = Field(..., description="Name of the supply chain being analyzed")
    date_created: date = Field(..., description="Date when the analysis was created")
    steps: List[SupplyChainStep] = Field(..., description="List of supply chain steps with calculated data")
    total_cycle_time: float = Field(..., description="Total cycle time for the entire supply chain (in hours)")
    total_throughput_rate: float = Field(..., description="Overall throughput rate of the supply chain (units per day)")
    total_inventory_cost: float = Field(..., description="Total inventory holding cost for the entire supply chain")
    total_stockout_cost: float = Field(..., description="Total stockout cost for the entire supply chain")
    total_cost: float = Field(..., description="Total cost for the entire supply chain")
    bottlenecks: List[str] = Field(..., description="List of identified bottlenecks in the supply chain")
    underutilized_resources: List[str] = Field(..., description="List of resources that are underutilized")
    optimization_recommendations: List[str] = Field(..., description="List of recommendations to optimize the supply chain performance")


In [16]:
from crewai_tools import tool

class CalculatorTools():

  @tool("Make a calculation")
  def calculate(operation: str) -> str:
    """Useful to perform any mathematical calculations,
    like sum, minus, multiplication, division, etc.
    The input to this tool should be a mathematical
    expression, a couple examples are `200*7` or `5000/2*10`
    """
    return eval(operation)

In [17]:
from crewai import Agent
from langchain_community.tools import TavilySearchResults
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper

# Initialize the Language Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(response_format={"type": "json_object"})

# Initialize tools
supply_chain_knowledge_base = GoogleSerperAPIWrapper()

# Tools for the agents
compiled_tools = [
    CalculatorTools.calculate,
    TavilySearchResults(
        max_results=3,
        include_answer=True,
        include_raw_content=True
    ),
    Tool(
        name="Bottleneck Identifier",
        func=supply_chain_knowledge_base.run,
        description="Identifies bottlenecks in the supply chain based on provided data."
    ),
    Tool(
        name="Cost Optimization Advisor",
        func=supply_chain_knowledge_base.run,
        description="Provides recommendations to reduce costs in the supply chain."
    ),
    Tool(
        name="Throughput Enhancer",
        func=supply_chain_knowledge_base.run,
        description="Suggests ways to improve throughput rates in the supply chain."
    ),
    Tool(
        name="Resource Utilization Analyzer",
        func=supply_chain_knowledge_base.run,
        description="Analyzes resource utilization to identify underutilized assets."
    ),
    Tool(
        name="Inventory Management Consultant",
        func=supply_chain_knowledge_base.run,
        description="Offers strategies for optimal inventory management."
    )
]

class SupplyChainManagementAgents:
    def data_collector(self):
        return Agent(
            role='Supply Chain Data Collector',
            goal="Accurately collect and validate initial data for each step in the supply chain, ensuring all inputs are thoroughly captured.",
            backstory="A meticulous supply chain analyst with expertise in comprehensive data gathering and validation.",
            verbose=True,
            tools=compiled_tools,
            llm=llm,
            memory=True
        )

    def data_analyzer(self):
        return Agent(
            role='Supply Chain Data Analyzer',
            goal="Analyze all collected data to calculate key metrics and identify bottlenecks, underutilized resources, and cost inefficiencies.",
            backstory="An analytical thinker skilled in supply chain metrics, performance evaluation, and thorough data analysis.",
            verbose=True,
            tools=compiled_tools,
            llm=llm,
            memory=True
        )

    def optimization_specialist(self):
        return Agent(
            role='Supply Chain Optimization Specialist',
            goal="Develop comprehensive recommendations to optimize the supply chain based on all analytical findings and input data.",
            backstory="An expert in supply chain optimization focused on enhancing efficiency, reducing costs, and improving resource utilization.",
            verbose=True,
            tools=compiled_tools,
            llm=llm,
            memory=True
        )

    def report_integrator(self):
        return Agent(
            role='Supply Chain Report Integrator',
            goal="Compile all analyses and recommendations into a detailed and professional supply chain analysis report, ensuring all inputs and findings are included.",
            backstory="A seasoned professional in reporting and presentation of complex supply chain analyses, adept at synthesizing data.",
            verbose=True,
            tools=compiled_tools,
            llm=llm,
            memory=True
        )


In [18]:
from crewai import Task
from textwrap import dedent

class SupplyChainManagementTasks:
    def collect_data(self, agent, input_data):
        return Task(
            description=dedent(f"""
                Collect and validate all initial data for the supply chain '{input_data.chain_name}' created on {input_data.date_created}.

                Ensure each step includes accurate and complete information:
                - Step name: {', '.join([step.step_name for step in input_data.steps])}
                - Step type: {', '.join([step.step_type for step in input_data.steps])}
                - Processing time: {', '.join([str(step.processing_time) for step in input_data.steps])} hours
                - Wait time: {', '.join([str(step.wait_time) for step in input_data.steps])} hours
                - Transportation time: {', '.join([str(step.transportation_time) for step in input_data.steps])} hours
                - Inventory level: {', '.join([str(step.inventory_level) for step in input_data.steps])}
                - Demand rate: {', '.join([str(step.demand_rate) for step in input_data.steps])} units/day
                - Supply rate: {', '.join([str(step.supply_rate) for step in input_data.steps])} units/day
                - Holding cost: {', '.join([str(step.holding_cost) for step in input_data.steps])} per unit/day
                - Stockout cost: {', '.join([str(step.stockout_cost) for step in input_data.steps])} per unit
                - Resources used: {', '.join([', '.join(step.resources) for step in input_data.steps])}

                Confirm that all data is accurate, consistent, and ready for comprehensive analysis.
                """),
            expected_output="""
                Confirmation that all supply chain data has been thoroughly collected and validated, with all inputs ready for analysis.
                """,
            agent=agent
        )

    def analyze_data(self, agent, input_data):
        return Task(
            description=dedent(f"""
                Analyze the validated data for the supply chain '{input_data.chain_name}'.

                For each step, perform the following calculations using all provided inputs:
                - Cycle time = Processing time + Wait time + Transportation time
                - Throughput rate = Minimum of supply rate and demand rate
                - Work-in-progress (WIP) = Throughput rate * Cycle time / 24 (to convert hours to days)
                - Total inventory holding cost = Inventory level * Holding cost
                - Total stockout cost = Stockout occurrences (if any) * Stockout cost
                - Total step cost = Total inventory holding cost + Total stockout cost
                - Utilization = (Processing time / Cycle time) * 100%

                Identify bottlenecks where the throughput rate is lower than other steps or where utilization is at or near 100%.

                Use the 'Bottleneck Identifier' tool to confirm bottlenecks, and 'Resource Utilization Analyzer' to identify underutilized resources.

                Structure the analysis according to the 'SupplyChainAnalysis' schema, including all calculated metrics, identified bottlenecks, and underutilized resources.
                """),
            expected_output="""
                A detailed analysis report with all calculated metrics using all input data, identifying bottlenecks and underutilized resources, structured according to the 'SupplyChainAnalysis' schema.
                """,
            agent=agent
        )

    def recommend_optimizations(self, agent, input_data):
        return Task(
            description=dedent(f"""
                Based on the comprehensive analysis of '{input_data.chain_name}', develop detailed optimization recommendations.

                For each bottleneck, underutilized resource, and cost inefficiency identified, use the relevant tools:
                - 'Cost Optimization Advisor' to suggest strategies for reducing holding and stockout costs.
                - 'Throughput Enhancer' to propose methods to improve throughput rates and reduce cycle times.
                - 'Inventory Management Consultant' to optimize inventory levels and balance supply and demand rates.
                - 'Resource Utilization Analyzer' to improve resource utilization rates.

                Ensure that recommendations are practical, data-driven, and directly address the specific issues identified in the analysis, utilizing all input data provided.

                Update the 'optimization_recommendations' field in the 'SupplyChainAnalysis' with these comprehensive suggestions.
                """),
            expected_output="""
                A list of actionable and detailed optimization recommendations included in the 'SupplyChainAnalysis', thoroughly addressing all identified issues using the full scope of input data.
                """,
            agent=agent
        )

    def integrate_report(self, agent, input_data):
        supply_chain_analysis_schema = SupplyChainAnalysis.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Compile a comprehensive report for the supply chain '{input_data.chain_name}'.

                The report should include:
                - Overview of the supply chain and date of analysis
                - Detailed steps with all calculated metrics from the analysis, including cycle time, throughput rate, WIP, total costs, and utilization rates
                - Total cycle time, total throughput rate, total inventory holding cost, total stockout cost, and total supply chain cost
                - Identified bottlenecks and underutilized resources with detailed explanations
                - Comprehensive optimization recommendations addressing all identified issues

                Ensure that all inputs and findings are included and clearly presented.

                Present the final report as valid JSON adhering strictly to the 'SupplyChainAnalysis' schema:
                {supply_chain_analysis_schema}
                """),
            expected_output="""
                A complete and detailed supply chain analysis report in JSON format, following the 'SupplyChainAnalysis' schema, integrating all inputs and findings from previous tasks.
                """,
            agent=agent
        )


In [19]:
from crewai import Crew

class SupplyChainManagementCrew:
    def __init__(self, input_data):
        self.input_data = input_data

    def run(self) -> str:
        agents = SupplyChainManagementAgents()
        tasks = SupplyChainManagementTasks()

        data_collector_agent = agents.data_collector()
        data_analyzer_agent = agents.data_analyzer()
        optimization_specialist_agent = agents.optimization_specialist()
        report_integrator_agent = agents.report_integrator()

        collect_data_task = tasks.collect_data(data_collector_agent, self.input_data)
        analyze_data_task = tasks.analyze_data(data_analyzer_agent, self.input_data)
        recommend_optimizations_task = tasks.recommend_optimizations(optimization_specialist_agent, self.input_data)
        integrate_report_task = tasks.integrate_report(report_integrator_agent, self.input_data)

        crew = Crew(
            agents=[
                data_collector_agent,
                data_analyzer_agent,
                optimization_specialist_agent,
                report_integrator_agent
            ],
            tasks=[
                collect_data_task,
                analyze_data_task,
                recommend_optimizations_task,
                integrate_report_task
            ],
            verbose=True
        )

        result = crew.kickoff()
        return result


In [20]:
if __name__ == "__main__":
    from datetime import datetime

    print("## Welcome to the Supply Chain Management Analysis Tool")
    print('------------------------------------------------------')

    # Input supply chain name and date
    chain_name = input("Enter the name of the supply chain: ")
    date_created_str = input("Enter the date of the analysis (YYYY-MM-DD): ")
    date_created = datetime.strptime(date_created_str, '%Y-%m-%d').date()

    # Input number of steps
    num_steps = int(input("Enter the number of steps in the supply chain: "))

    steps = []
    for i in range(num_steps):
        print(f"\nEnter details for Step {i+1}:")
        step_name = input("Step name: ")
        step_type = input("Step type (Procurement/Manufacturing/Distribution/Retail): ")
        processing_time = float(input("Processing time (hours): "))
        wait_time = float(input("Wait time before step begins (hours): "))
        transportation_time = float(input("Transportation time (hours): "))
        inventory_level = int(input("Inventory level at this step: "))
        demand_rate = int(input("Demand rate at this step (units/day): "))
        supply_rate = int(input("Supply rate at this step (units/day): "))
        holding_cost = float(input("Holding cost per unit per day: "))
        stockout_cost = float(input("Stockout cost per unit: "))
        resources_input = input("Resources used (comma-separated): ")
        resources = [res.strip() for res in resources_input.split(',')]

        step = SupplyChainStepInput(
            step_name=step_name,
            step_type=step_type,
            processing_time=processing_time,
            wait_time=wait_time,
            transportation_time=transportation_time,
            inventory_level=inventory_level,
            demand_rate=demand_rate,
            supply_rate=supply_rate,
            holding_cost=holding_cost,
            stockout_cost=stockout_cost,
            resources=resources
        )
        steps.append(step)

    input_data = SupplyChainInputData(
        chain_name=chain_name,
        date_created=date_created,
        steps=steps
    )

    scm_crew = SupplyChainManagementCrew(input_data)

    result = scm_crew.run()
    print("\n\n########################################")
    print("## Comprehensive Supply Chain Report")
    print("########################################\n")
    print(result)


## Welcome to the Supply Chain Management Analysis Tool
------------------------------------------------------
Enter the name of the supply chain: Global Electronics Supply Chain
Enter the date of the analysis (YYYY-MM-DD): 2024-10-14
Enter the number of steps in the supply chain: 3

Enter details for Step 1:
Step name: Raw Material Procurement
Step type (Procurement/Manufacturing/Distribution/Retail): Procurement
Processing time (hours): 48.0
Wait time before step begins (hours): 12.0
Transportation time (hours): 24.0
Inventory level at this step: 5000
Demand rate at this step (units/day): 200
Supply rate at this step (units/day): 250
Holding cost per unit per day: 0.50
Stockout cost per unit: 100.0
Resources used (comma-separated): Steel, Copper, Plastic

Enter details for Step 2:
Step name: Component Manufacturing
Step type (Procurement/Manufacturing/Distribution/Retail): Manufacturing
Processing time (hours): 72.0
Wait time before step begins (hours): 8.0
Transportation time (hours

# Agent: Supply Chain Data Collector
## Task: 
Collect and validate all initial data for the supply chain 'Global Electronics Supply Chain' created on 2024-10-14.

Ensure each step includes accurate and complete information:
- Step name: Raw Material Procurement, Component Manufacturing, Final Product Distribution
- Step type: Procurement, Manufacturing, Distribution
- Processing time: 48.0, 72.0, 24.0 hours
- Wait time: 12.0, 8.0, 5.0 hours
- Transportation time: 24.0, 10.0, 48.0 hours
- Inventory level: 5000, 3000, 1500
- Demand rate: 200, 150, 300 units/day
- Supply rate: 250, 180, 350 units/day
- Holding cost: 0.5, 0.75, 1.0 per unit/day
- Stockout cost: 100.0, 80.0, 50.0 per unit
- Resources used: Steel, Copper, Plastic, Machines, Labor, Tools, Trucks, Warehouses

Confirm that all data is accurate, consistent, and ready for comprehensive analysis.



# Agent: Supply Chain Data Collector
## Thought: I need to collect and validate the initial data for the 'Global Electronics Supply 

In [21]:
result.raw

'{\n  "chain_name": "Global Electronics Supply Chain",\n  "date_created": "2023-10-10",\n  "steps": [\n    {\n      "step_name": "Procurement",\n      "step_type": "Procurement",\n      "processing_time": 5,\n      "wait_time": 2,\n      "transportation_time": 1,\n      "inventory_level": 100,\n      "demand_rate": 50,\n      "supply_rate": 60,\n      "holding_cost": 0.5,\n      "stockout_cost": 10,\n      "resources": ["Supplier A", "Supplier B"],\n      "cycle_time": 8,\n      "throughput_rate": 50,\n      "work_in_progress": 20,\n      "total_inventory_cost": 50,\n      "total_stockout_cost": 0,\n      "total_step_cost": 50,\n      "utilization": 83.33\n    },\n    {\n      "step_name": "Manufacturing",\n      "step_type": "Manufacturing",\n      "processing_time": 10,\n      "wait_time": 3,\n      "transportation_time": 2,\n      "inventory_level": 200,\n      "demand_rate": 80,\n      "supply_rate": 100,\n      "holding_cost": 0.75,\n      "stockout_cost": 15,\n      "resources": 

In [22]:
import json
data = json.loads(result.raw)
data

{'chain_name': 'Global Electronics Supply Chain',
 'date_created': '2023-10-10',
 'steps': [{'step_name': 'Procurement',
   'step_type': 'Procurement',
   'processing_time': 5,
   'wait_time': 2,
   'transportation_time': 1,
   'inventory_level': 100,
   'demand_rate': 50,
   'supply_rate': 60,
   'holding_cost': 0.5,
   'stockout_cost': 10,
   'resources': ['Supplier A', 'Supplier B'],
   'cycle_time': 8,
   'throughput_rate': 50,
   'work_in_progress': 20,
   'total_inventory_cost': 50,
   'total_stockout_cost': 0,
   'total_step_cost': 50,
   'utilization': 83.33},
  {'step_name': 'Manufacturing',
   'step_type': 'Manufacturing',
   'processing_time': 10,
   'wait_time': 3,
   'transportation_time': 2,
   'inventory_level': 200,
   'demand_rate': 80,
   'supply_rate': 100,
   'holding_cost': 0.75,
   'stockout_cost': 15,
   'resources': ['Factory A', 'Machine 1'],
   'cycle_time': 15,
   'throughput_rate': 80,
   'work_in_progress': 50,
   'total_inventory_cost': 150,
   'total_stoc